In [15]:
#%%
import math

'''
Input some parameters 
such as PIM array size, network.
'''

array = [512, 512]


network = 'Resnet18'

if network == 'VGG13' :
  image = [224, 224, 112, 112, 56, 56, 28, 28, 14, 14]
  kernel = [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
  channel = [3, 64, 64, 128, 128, 256, 256, 512, 512, 512, 512]

elif network == 'Resnet18' :
  image = [112, 56, 28, 14, 7]
  kernel = [7, 3, 3, 3, 3]
  channel = [3, 64, 128, 256, 512]
# elif network == 'SqueezeNet':
#   image = [224, 55, 27, 13, 13]
#   kernel = [3, 3, 3, 3, 3]
#   channel = [3, 16, 32, 48, 64]
# elif network == 'Lenet-5':
#   image = [32, 14, 5]
#   kernel = [5, 5, 5]
#   channel = [1, 6, 16]

max_iterations = 1000
iterations_count = 0

# %%
def im2col (image_col, image_row, filter_col, filter_row, in_channel, out_channel, array_row, array_col) :

    col_slide = image_col - filter_col + 1
    row_slide = image_row - filter_row + 1
    
    col_cycle = math.ceil(out_channel/array_col)
    row_cycle = math.ceil(filter_row*filter_col*in_channel/array_row)
    total_cycle = col_slide * row_slide * row_cycle * col_cycle
    
    return total_cycle


def SDK (image_col, image_row, filter_col, filter_row, in_channel, out_channel, \
                    array_row, array_col) :
    
    row_vector = filter_row * filter_col * in_channel
    col_vector = out_channel
    
    used_row = math.ceil(row_vector/array_row)
    used_col = math.ceil(col_vector/array_col)
    
    new_array_row = array_row * used_row
    new_array_col = array_col * used_col

    # initialize
    cycle = []
    w = []
    w.append(filter_row*filter_col)
    cycle.append(used_row*used_col*(image_row-filter_row+1)*(image_col-filter_col+1))
    
    i=0
    while True :
        i += 1
        pw_row = filter_row + i - 1 
        pw_col = filter_col + i - 1
        pw = pw_row * pw_col
        if pw*in_channel <= new_array_row and i * i * out_channel <= new_array_col :
            parallel_window_row = math.ceil((image_row - (filter_row + i) + 1)/i) + 1
            parallel_window_col = math.ceil((image_col - (filter_col + i) + 1)/i) + 1
            
            if parallel_window_row * parallel_window_row * used_row * used_col <= cycle[0] :
                del cycle[0]
                del w[0]
                cycle.append(parallel_window_row * parallel_window_col * used_row * used_col)
                w.append(pw)
            
        else :
            break
        
    
    return cycle, w

# ceil : up, floor : down
def vw_sdk (image_col, image_row, filter_col, filter_row, in_channel, out_channel, \
                    array_row, array_col) :

    i = 0 # initialize # overlap col
    j = 1 # overlap row

    reg_total_cycle = [] # initialize
    reg_overlap_row = []
    reg_overlap_col = []
    reg_row_cycle = []
    reg_col_cycle = []
    reg_ICt = []
    reg_OCt = []
    
    data_tran = []
    iterations_count = 0
    while True :
        try :
            i += 1
            if (i + filter_col) > image_col: 
                i = 1
                j += 1
                if j + filter_row > image_row : 
                    break


            # for parallel_window computing
            reg_N_parallel_window_row = math.ceil((image_row - (filter_row + i) + 1)/i) + 1
            reg_N_parallel_window_col = math.ceil((image_col - (filter_col + j) + 1)/j) + 1
            
            # for cycle computing
            # Tiled IC
            if in_channel == 3 :
                ICt = math.floor(array_row /((filter_row + i - 1)*(filter_col + j - 1)))
                if ICt > in_channel :
                    ICt = 3
                row_cycle = math.ceil(in_channel / ICt)
            else :
                ICt = math.floor(array_row /((filter_row + i - 1)*(filter_col + j - 1)))
                row_cycle = math.ceil(in_channel / ICt)
            
            # Tiled OC
            OCt =  math.floor(array_col / (i * j))
            col_cycle = math.ceil(out_channel / OCt)
    
            reg_N_of_computing_cycle = reg_N_parallel_window_row * reg_N_parallel_window_col \
                                    * row_cycle * col_cycle
            # //trand_fre  data_num          
            out_imagerow = (image_row - filter_row+1)
            out_imagecol = (image_col - filter_col+1)
            outrow_per_calculate = i 
            outcol_per_calculate = j 
            # total_num_trans = math.ceil(out_imagerow/outrow_per_calculate)*math.ceil(out_imagecol/outcol_per_calculate)

            trans_fre = ICt*(filter_col+i)*(filter_row+j)
            data_num = reg_N_parallel_window_row * reg_N_parallel_window_col * trans_fre * row_cycle
            if i == 1 : # initialize
                reg_total_cycle.append(reg_N_of_computing_cycle)
                reg_overlap_row.append(i)
                reg_overlap_col.append(j)
                reg_row_cycle.append(row_cycle)
                reg_col_cycle.append(col_cycle)
                reg_ICt.append(ICt)
                reg_OCt.append(OCt)
                data_tran.append(data_num)

            if reg_total_cycle[0] > reg_N_of_computing_cycle :
                del reg_total_cycle[0]
                del reg_overlap_row[0]
                del reg_overlap_col[0]
                del reg_row_cycle[0]
                del reg_col_cycle[0]
                del reg_ICt[0]
                del reg_OCt[0]
                del data_tran[0]

                reg_total_cycle.append(reg_N_of_computing_cycle)
                reg_overlap_row.append(i)
                reg_overlap_col.append(j)
                reg_row_cycle.append(row_cycle)
                reg_col_cycle.append(col_cycle)
                reg_ICt.append(ICt)
                reg_OCt.append(OCt)
                data_tran.append(data_num)
            # if reg_total_cycle[0]+data_tran[0] > reg_N_of_computing_cycle+data_num:
            #   del reg_total_cycle[0]
            #   del reg_overlap_row[0]
            #   del reg_overlap_col[0]
            #   del reg_row_cycle[0]
            #   del reg_col_cycle[0]
            #   del reg_ICt[0]
            #   del reg_OCt[0]
            #   del data_tran[0]

            #   reg_total_cycle.append(reg_N_of_computing_cycle)
            #   reg_overlap_row.append(i)
            #   reg_overlap_col.append(j)
            #   reg_row_cycle.append(row_cycle)
            #   reg_col_cycle.append(col_cycle)
            #   reg_ICt.append(ICt)
            #   reg_OCt.append(OCt)
            #   data_tran.append(data_num)
            # print("end while")
        except ZeroDivisionError :
            continue

    return reg_total_cycle[0], reg_overlap_col[0], reg_overlap_row[0], reg_row_cycle[0], reg_col_cycle[0], reg_ICt[0], reg_OCt[0], data_tran[0] 

def network_information(network, image) :
    print("="*50)
    print(" Network : ", network)
    print( " Array Size = {} x {}".format(array[0], array[1]))
    print("-"*30)
    
    print(" NETWORK INFORMATION ")
    print("-"*30)
    for i in range(len(image)) :
        print(" CONV LAYER "+ str(i+1))
        print("    Image   Size = {} x {}".format(image[i], image[i]))
        print("    Kernel  Size = {} x {}".format(kernel[i], kernel[i]))
        if network == 'VGG13' :
          print("    Channel Size = {} x {}".format(channel[i], channel[i+1]))
        elif network == 'Resnet13' :
          print("    Channel Size = {} x {}".format(channel[i], channel[i]))
    print("="*50)
    
def result (network, image) :
    CC_im2col = []
    
    CC = []
    SDK_height = []
    SDK_width = []
    AR_cycle = []
    AC_cycle = []
    IC_tiled = []
    OC_tiled = []
    data_transmisson = []
    
    CC_SDK = []
    PW_SDK = []
    
    print("="*50)
    print(" RESULTS of COMPUTING CYCLES")
    print("-"*30)

    if network == 'VGG13' :
      
      for i in range(len(image)) :
          T_im2col = im2col(image[i], image[i], kernel[i], kernel[i], channel[i], channel[i+1], array[0], array[1])
          CC_im2col.append(T_im2col)

          T_SDK, w = SDK(image[i], image[i], kernel[i], kernel[i], channel[i], channel[i+1], array[0], array[1])
          CC_SDK.append(T_SDK[0])
          PW_SDK.append(w[0])

          T_cycle, SDK_h, SDK_w, ARC, ACC, tiled_IC, tiled_OC, data_flow = vw_sdk(image[i], image[i], kernel[i], kernel[i], channel[i], channel[i+1], array[0], array[1])
          CC.append(T_cycle)
          SDK_height.append(SDK_h)
          SDK_width.append(SDK_w)
          AR_cycle.append(ARC)
          AC_cycle.append(ACC)
          IC_tiled.append(tiled_IC)
          OC_tiled.append(tiled_OC)
          data_transmisson.append(data_flow)

      for i in range(len(image)) :
        # print(" CONV LAYER "+ str(i+1))
        # print("    Im2col = {}".format(CC_im2col[i]))
        # print("    SDK    = {}".format(CC_SDK[i]))
        # if i == 0 :
        #   print("      - shape of PW = {} x {} x {} x {}".format(int(math.sqrt(PW_SDK[i])), int(math.sqrt(PW_SDK[i])), channel[i], channel[i+1]))
        # else :
        #   print("      - shape of PW = {} x {} x {} x {}".format(int(math.sqrt(PW_SDK[i])), int(math.sqrt(PW_SDK[i])), channel[i], channel[i+1]))

        if CC[i] >= CC_im2col[i] :
          CC[i] = CC_im2col[i]
          print("    VW-SDK = {}".format(CC[i]))
          print("      - Optimal shape of PW = {} x {} x {} x {}".format(kernel[i], kernel[i], channel[i], channel[i+1]))
          print("      - Reduction Compared to Im2col = {:.2f} %".format((CC_im2col[i]-CC[i])/CC_im2col[i]*100))
          print("      - Reduction Compared to SDK    = {:.2f} %".format((CC_SDK[i]-CC[i])/CC_SDK[i]*100))    

        else :
          print("    VW-SDK = {}".format(CC[i]))
          print("      - Optimal shape of PW = {} x {} x {} x {}".format(kernel[i] + SDK_width[i]-1, kernel[i] + SDK_height[i]-1, IC_tiled[i], OC_tiled[i]))
          print("      - Reduction Compared to Im2col = {:.2f} %".format((CC_im2col[i]-CC[i])/CC_im2col[i]*100))
          print("      - Reduction Compared to SDK    = {:.2f} %".format((CC_SDK[i]-CC[i])/CC_SDK[i]*100))    
      print("="*50)
    
    elif network == 'Resnet18' :

      for i in range(len(image)) :
        if i == 0 :
          T_im2col = im2col(image[i], image[i], kernel[i], kernel[i], channel[i], channel[i+1], array[0], array[1])
          T_SDK, w = SDK(image[i], image[i], kernel[i], kernel[i], channel[i], channel[i+1], array[0], array[1])
          T_cycle, SDK_h, SDK_w, ARC, ACC, tiled_IC, tiled_OC, data_flow = vw_sdk(image[i], image[i], kernel[i], kernel[i], channel[i], channel[i+1], array[0], array[1])
        
        else :
          T_im2col = im2col(image[i], image[i], kernel[i], kernel[i], channel[i], channel[i], array[0], array[1])
          T_SDK, w = SDK(image[i], image[i], kernel[i], kernel[i], channel[i], channel[i], array[0], array[1])
          T_cycle, SDK_h, SDK_w, ARC, ACC, tiled_IC, tiled_OC, data_flow = vw_sdk(image[i], image[i], kernel[i], kernel[i], channel[i], channel[i], array[0], array[1])
          
        CC_im2col.append(T_im2col)
        
        CC_SDK.append(T_SDK[0])
        PW_SDK.append(w[0])          
        
        CC.append(T_cycle)
        SDK_height.append(SDK_h)
        SDK_width.append(SDK_w)
        AR_cycle.append(ARC)
        AC_cycle.append(ACC)
        IC_tiled.append(tiled_IC)
        OC_tiled.append(tiled_OC)
        data_transmisson.append(data_flow)

      for i in range(len(image)) :
        # print(" CONV LAYER "+ str(i+1))
        # print("    Im2col = {}".format(CC_im2col[i]))
        # print("    SDK    = {}".format(CC_SDK[i]))
        # if i == 0 :
        #   print("      - shape of PW = {} x {} x {} x {}".format(int(math.sqrt(PW_SDK[i])), int(math.sqrt(PW_SDK[i])), channel[i], channel[i+1]))
        # else :
        #   print("      - shape of PW = {} x {} x {} x {}".format(int(math.sqrt(PW_SDK[i])), int(math.sqrt(PW_SDK[i])), channel[i], channel[i]))

        if CC[i] > CC_im2col[i] :
          CC[i] = CC_im2col[i]
          print("    VW-SDK-W = {}".format(SDK_width[i]))
          print("    VW-SDK-H = {}".format(SDK_height[i]))
          print("    VW-SDK-Data = {}".format(data_transmisson[i]))
          print("    VW-SDK = {}".format(CC[i]))
          if i == 0 :
            print("      - Optimal shape of PW = {} x {} x {} x {}".format(kernel[i], kernel[i], channel[i], channel[i+1]))
          else :
            print("      - Optimal shape of PW = {} x {} x {} x {}".format(kernel[i], kernel[i], channel[i], channel[i]))
          print("      - Reduction Compared to Im2col = {:.2f} %".format((CC_im2col[i]-CC[i])/CC_im2col[i]*100))
          print("      - Reduction Compared to SDK    = {:.2f} %".format((CC_SDK[i]-CC[i])/CC_SDK[i]*100))    

        else :
          print("    VW-SDK-W = {}".format(SDK_width[i]))
          print("    VW-SDK-H = {}".format(SDK_height[i]))
          print("    VW-SDK = {}".format(data_transmisson[i]))
          print("    VW-SDK = {}".format(CC[i]))
          print("      - Optimal shape of PW = {} x {} x {} x {}".format(kernel[i] + SDK_width[i]-1, kernel[i] + SDK_height[i]-1, IC_tiled[i], OC_tiled[i]))
          print("      - Reduction Compared to Im2col = {:.2f} %".format((CC_im2col[i]-CC[i])/CC_im2col[i]*100))
          print("      - Reduction Compared to SDK    = {:.2f} %".format((CC_SDK[i]-CC[i])/CC_SDK[i]*100))    
      print("="*50)
    
# %%
network_information(network, image)
# result(network, image)

 Network :  Resnet18
 Array Size = 512 x 512
------------------------------
 NETWORK INFORMATION 
------------------------------
 CONV LAYER 1
    Image   Size = 112 x 112
    Kernel  Size = 7 x 7
 CONV LAYER 2
    Image   Size = 56 x 56
    Kernel  Size = 3 x 3
 CONV LAYER 3
    Image   Size = 28 x 28
    Kernel  Size = 3 x 3
 CONV LAYER 4
    Image   Size = 14 x 14
    Kernel  Size = 3 x 3
 CONV LAYER 5
    Image   Size = 7 x 7
    Kernel  Size = 3 x 3


In [16]:
result(network, image)

 RESULTS of COMPUTING CYCLES
------------------------------
    VW-SDK-W = 4
    VW-SDK-H = 2
    VW-SDK = 425007
    VW-SDK = 1431
      - Optimal shape of PW = 10 x 8 x 3 x 64
      - Reduction Compared to Im2col = 87.26 %
      - Reduction Compared to SDK    = 49.06 %
    VW-SDK-W = 2
    VW-SDK-H = 2
    VW-SDK = 1166400
    VW-SDK = 1458
      - Optimal shape of PW = 4 x 4 x 32 x 128
      - Reduction Compared to Im2col = 75.00 %
      - Reduction Compared to SDK    = 0.00 %
    VW-SDK-W = 2
    VW-SDK-H = 2
    VW-SDK = 540800
    VW-SDK = 676
      - Optimal shape of PW = 4 x 4 x 32 x 128
      - Reduction Compared to Im2col = 66.67 %
      - Reduction Compared to SDK    = 66.67 %
    VW-SDK-W = 2
    VW-SDK-H = 1
    VW-SDK = 423360
    VW-SDK = 504
      - Optimal shape of PW = 4 x 3 x 42 x 256
      - Reduction Compared to Im2col = 30.00 %
      - Reduction Compared to SDK    = 30.00 %
    VW-SDK-W = 1
    VW-SDK-H = 1
    VW-SDK-Data = 224000
    VW-SDK = 225
      - Optimal

In [46]:
import math
# calculate conpute cycle and transmit data
def calculate_performance(image_row, image_col, kernel, inchannel, outchannel, PW_h, PW_w, ICt, OCt, array_row, array_col):
    # reg transmit data
    flexible_window = []
    i = PW_h-kernel+1
    j = PW_w-kernel+1
    row_outCount = math.ceil((image_row-PW_h)/(PW_h-kernel+1))+1
    col_outCount = math.ceil((image_col-PW_w)/(PW_w-kernel+1))+1
    total_transfer = row_outCount*col_outCount
    num_ic = math.ceil(inchannel/ICt)
    # num_oc = math.ceil(outchannel/OCt)
    totalTransferredData = total_transfer*num_ic*PW_h*PW_w*ICt
    #reg compute cycle
    AR_cycle = math.ceil(inchannel*PW_h*PW_w/array_row)
    AC_cycle = math.ceil(OCt*i*j/array_col)*math.ceil(outchannel/OCt)
    total_compute_cycle = total_transfer*AR_cycle*AC_cycle
    #reg array num
    flexible_window.append(PW_h)
    flexible_window.append(PW_w)
    flexible_window.append(ICt)
    flexible_window.append(OCt)
    total_array = AC_cycle*AR_cycle
    return totalTransferredData, total_compute_cycle, total_array, flexible_window

#
calculate_performance(112,112,7,3,64,10,8,3,64,512,512)

(343440, 1431, 1, [10, 8, 3, 64])

In [70]:
import csv
# Read neural network data
# Split convolutional layers and fully connected layers
def read_csv(file_path):
    conv_layers = []
    fc_layers = []
    
    with open(file_path, 'r') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            float_row = [float(value) for value in row]
            if float_row[3]==1:
                fc_layers.append(float_row)
            else:
                conv_layers.append(float_row)
    return conv_layers,fc_layers

# Computational neural networks require arrays
def calculate_min_array(file_path, array_row, array_col):
    conv, fc = read_csv(file_path)
    conv_array = 0
    fc_array = 0
    for i in range(len(conv)):
        row = conv[i][2]*conv[i][3]*conv[i][4]
        col = conv[i][5]
        conv_row_num = math.ceil(row/array_row)
        conv_col_num = math.ceil(col/array_col)
        conv_array = conv_row_num*conv_col_num + conv_array
    for j in range(len(fc)):
        fc_row_num = math.ceil(fc[j][2]/array_row)
        fc_col_num = math.ceil(fc[j][5]/array_col)
        fc_array = fc_array + fc_col_num*fc_row_num
    return conv_array, fc_array



#Split the data in the list
def split_listdata(data_list, index):
    data1 = []
    data2 = []
    for item in data_list:
        data1.append(item[:index])
        data2.append(item[index:])
    return data1, data2

# pareto
def is_dominated(solution1, solution2):
        """
        判断solution1是否被solution2支配
        :param solution1: 第一个解
        :param solution2: 第二个解
        :return: 如果solution1被solution2支配,返回True,否则返回False
        """
        # 检查solution2是否在所有指标上都优于solution1
        for i in range(len(solution1)):
            if (solution2[0] <= solution1[0] and
                solution2[1] <= solution1[1] and
                solution2[2] <= solution1[2]):
                # 至少有一个指标更优
                if (solution2[0] < solution1[0] or
                    solution2[1] < solution1[1] or
                    solution2[2] < solution1[2]):
                    return True
        return False

def update_pareto_set(new_solution, pareto_set):
    """
    更新帕累托最优解集
    :param new_solution: 新计算出的解（字典形式，包含三个指标）
    :param pareto_set: 当前的帕累托最优解集（列表形式）
    :return: 更新后的帕累托最优解集
    """
    # 检查新解是否被现有解集中的解支配
    dominated_by_existing = False
    if len(pareto_set)==0:
        pareto_set.append(new_solution)
        return pareto_set
    else:
        for solution in pareto_set:
            if is_dominated(new_solution, solution):
                dominated_by_existing = True
                break
        
        if not dominated_by_existing:
            # 移除被新解支配的解
            new_pareto_set = []
            for solution in pareto_set:
                if not is_dominated(solution, new_solution):
                    new_pareto_set.append(solution)
            new_pareto_set.append(new_solution)
            return new_pareto_set
        else:
            return pareto_set

def auto_mapping(image_row, image_col, kernel, inchannel, outchannel, array_row, array_col):
    PW_h = kernel
    PW_w = kernel
    # reg pareto set
    s1 = []
    s2 = []
    pareto = []
    pareto_performance = []
    pareto_design = []
    flex_window = []

    for i in range(image_row - kernel):
        for j in range(image_col - kernel):
            PW_h = kernel + i
            PW_w = kernel + j
            ICt = max(math.floor(array_row/PW_h/PW_w),1)
            OCt = max(math.floor(array_col/(i+1)/(j+1)),1)
            if OCt>=outchannel:
                OCt = outchannel
            if ICt>=inchannel:
                ICt = inchannel
            total_transferdata, total_computecycle, total_array, flex_window = calculate_performance(image_row, image_row, kernel, inchannel, outchannel,PW_h, PW_w, ICt, OCt, array_row, array_col)
            s1.append(total_transferdata)
            s1.append(total_computecycle)
            s1.append(total_array)
            s1.append(flex_window)
            s2 = s1[:]
            pareto = update_pareto_set(s2,pareto)
            pareto_performance, pareto_design = split_listdata(pareto, 3)
            s1.clear()
            # print("The total amount of data transmitted :",total_transferdata)
            # print("Total calculation cycle :",total_computecycle)
            # print("The total array :",total_array)

    return pareto_performance,pareto_design


# def Automated_Exploration_Framework(Network, array_row, array_col, max_numarray):
#     minnum_array = calculate_min_array(Network,array_row,array_col)
#     # If it is smaller than the minimum array, special mapping is required
#     # If greater than or equal to the minimum array, automatic mapping search is required
#     if minnum_array < max_numarray:

#     else:

performance, design=auto_mapping(112,112,7,3,64,512,512)
print(len(performance))


58


In [71]:
performance

[[343440, 1431, 1],
 [216240, 1590, 3],
 [202176, 1728, 3],
 [343440, 1431, 1],
 [218700, 1458, 2],
 [196020, 1782, 3],
 [158760, 1512, 4],
 [126360, 3564, 22],
 [196020, 1782, 3],
 [156816, 1584, 4],
 [130680, 1584, 6],
 [156816, 1584, 4],
 [139968, 1620, 5],
 [127008, 1764, 7],
 [99792, 3276, 26],
 [202176, 1728, 3],
 [158760, 1512, 4],
 [127008, 1764, 7],
 [130680, 1584, 6],
 [122400, 3120, 26],
 [86400, 3072, 32],
 [77760, 4752, 66],
 [85536, 3168, 32],
 [216240, 1590, 3],
 [122400, 3120, 26],
 [81600, 3520, 44],
 [85536, 3168, 32],
 [78732, 3564, 44],
 [77760, 4752, 66],
 [68040, 8640, 192],
 [86400, 3072, 32],
 [81600, 3520, 44],
 [77760, 4752, 66],
 [69120, 4608, 96],
 [99792, 3276, 26],
 [77760, 4752, 66],
 [69120, 4608, 96],
 [60480, 7680, 256],
 [57024, 7680, 320],
 [54432, 13824, 768],
 [126360, 3564, 22],
 [68040, 8640, 192],
 [60480, 7680, 256],
 [52920, 13440, 896],
 [49560, 19200, 1920],
 [57024, 7680, 320],
 [52272, 14336, 896],
 [49896, 13824, 1152],
 [46728, 18432, 23

In [72]:
calculate_performance(224,224,3,3,64,10,3,3,64,512,512)

(559440, 6216, 1, [10, 3, 3, 64])

In [73]:
calculate_performance(224,224,3,3,64,4,6,3,64,512,512)

(447552, 6216, 1, [4, 6, 3, 64])

In [74]:
calculate_min_array('NetWork_Resnet20.csv',512,512)

(24, 1)

In [76]:
conv, fc=read_csv('NetWork_Resnet20.csv')
fc

[[1.0, 1.0, 64.0, 1.0, 1.0, 10.0, 0.0, 0.0]]